# Traducción a Python 
A continuación, en el archivo tiempo_respuesta.R se presenta una función de cálculo de tiempos de respuesta de solicitudes de clientes, esta función esta escrita en lenguaje R. Traduzca la función a lenguaje Python.

In [ ]:
Preguntas

- Asumo que es del tipo YYYY-MM-DD HH-MM-SS
- Formato de x: dataframe o dict????

## Importar Librerias

In [115]:
import pandas as pd
import numpy as np
from datetime import datetime, date

# Conexion a Google Drive

In [2]:
#Ejecuta la celda y sigue los pasos, es necesario conectarse con la cuenta de usuario  en la que se encuentran los archivos.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Declarar directorios

In [3]:
source_Dir = '/content/drive/MyDrive/Desafío GM/data/Calendario.xlsx' # Ruta a archivo xlsx.

# Traducción R - Python

In [90]:
# Crear Dataframe de prueba.
x = pd.DataFrame({"CREATEDON" : ["02/01/2017 09:06:10", "05/01/2017 09:06:10"],
                  "UPDATEDON" : ["07/01/2017 11:10:00", "09/01/2017 09:06:10"]})
# Change register formats to datetime.
for register in x:
  x[register] = pd.to_datetime(x[register], format='%d/%m/%Y %H:%M:%S')

x["CREATEDON_AUX"] = x["CREATEDON"] 
x["UPDATEDON_AUX"] = x["UPDATEDON"] 
x

,CREATEDON,UPDATEDON,CREATEDON_AUX,UPDATEDON_AUX
0,2017-01-02 09:06:10,2017-01-07 11:10:00,2017-01-02 09:06:10,2017-01-07 11:10:00
1,2017-01-05 09:06:10,2017-01-09 09:06:10,2017-01-05 09:06:10,2017-01-09 09:06:10


In [91]:
#Get the delta time and transform it to seconds.
x['TiempoRespuesta_2'] = pd.to_timedelta(x['UPDATEDON'] - x['CREATEDON']).dt.total_seconds()
x['TiempoRespuesta_2']

0    439430.0
1    345600.0
Name: TiempoRespuesta_2, dtype: float64

In [92]:
x.columns

Index(['CREATEDON', 'UPDATEDON', 'CREATEDON_AUX', 'UPDATEDON_AUX',
       'TiempoRespuesta_2'],
      dtype='object')

In [93]:
###### 
## CALCULO REAL DEL TIEMPO DE SOLICITUDES
# Obtiene las fechas de creacion y respuesta de consulta.
x["day_created"]= x["CREATEDON_AUX"].dt.normalize()
x["day_answer"] = x["UPDATEDON_AUX"].dt.normalize()
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   CREATEDON          2 non-null      datetime64[ns]
 1   UPDATEDON          2 non-null      datetime64[ns]
 2   CREATEDON_AUX      2 non-null      datetime64[ns]
 3   UPDATEDON_AUX      2 non-null      datetime64[ns]
 4   TiempoRespuesta_2  2 non-null      float64       
 5   day_created        2 non-null      datetime64[ns]
 6   day_answer         2 non-null      datetime64[ns]
dtypes: datetime64[ns](6), float64(1)
memory usage: 240.0 bytes


In [94]:
# Función para encontrar el nombre del día de ingreso y respuesta de la solicitud.
def get_name_of_day(x, calendario, date_generated, column_name):
  # Cambiar el nombre de las columnas con el número de índice.
  calendario.rename(columns={calendario.columns[0] : date_generated,
                  calendario.columns[1] : column_name}, inplace=True)
  #Conversión de columna de formato de objeto a fecha y hora.
  calendario[date_generated] = pd.to_datetime(calendario[date_generated], format="%d/%m/%Y")
  #Fusionar marcos de datos x y calendario por la columna "day_created".
  x = pd.merge(x,calendario, on=(date_generated) ,how='left')
  return x

In [95]:
#Obtener Dataframe con fechas y dias de la semana (por nombre) desde el calendario.
calendario = pd.read_excel(source_Dir, usecols=[0,3])

x = get_name_of_day(x, calendario, "day_created", "DIA_SEMANA_CREATED")
x = get_name_of_day(x, calendario, "day_answer", "DIA_SEMANA_ANSWER")
x

,CREATEDON,UPDATEDON,CREATEDON_AUX,UPDATEDON_AUX,TiempoRespuesta_2,day_created,day_answer,DIA_SEMANA_CREATED,DIA_SEMANA_ANSWER
0,2017-01-02 09:06:10,2017-01-07 11:10:00,2017-01-02 09:06:10,2017-01-07 11:10:00,439430.0,2017-01-02,2017-01-07,Lunes,Sábado
1,2017-01-05 09:06:10,2017-01-09 09:06:10,2017-01-05 09:06:10,2017-01-09 09:06:10,345600.0,2017-01-05,2017-01-09,Jueves,Lunes


In [96]:
# Encontrar la hora de inicio del dia laboral en el cual se genero el registro.
def get_hora_inicio_dia(DIA_AUX, DIA_SEMANA="Sábado"):
  #Posibles horas de inicio según el día.
  info_dict = {
    "Sábado": "09:30:00",
    "Domingo": "11:00:00",
  }
  DIA = DIA_AUX.strftime("%m/%d/%Y %H:%M:%S").split(' ')[0]
  #El horario por defecto corresponde a la entrada de un día hábil (08:30:00).
  HORA_COMIENZO = info_dict.get(DIA_SEMANA, "08:30:00")

  INICIO_CREATED = pd.to_datetime(f'{DIA} {HORA_COMIENZO}', format='%d/%m/%Y %H:%M:%S')
  return INICIO_CREATED

In [97]:
# Encontrar la hora de inicio del dia laboral en el cual se genero la peticion.
x["INICIO_CREATED"] =  x.apply(lambda y: get_hora_inicio_dia(y['CREATEDON_AUX'], y['DIA_SEMANA_CREATED']), axis=1)
# Encontrar la hora de inicio del dia laboral en el cual se genero la respuesta.
x["INICIO_ANSWERED"] = x.apply(lambda y: get_hora_inicio_dia(y['UPDATEDON_AUX'], y['DIA_SEMANA_ANSWER']), axis=1)
x

,CREATEDON,UPDATEDON,CREATEDON_AUX,UPDATEDON_AUX,TiempoRespuesta_2,day_created,day_answer,DIA_SEMANA_CREATED,DIA_SEMANA_ANSWER,INICIO_CREATED,INICIO_ANSWERED
0,2017-01-02 09:06:10,2017-01-07 11:10:00,2017-01-02 09:06:10,2017-01-07 11:10:00,439430.0,2017-01-02,2017-01-07,Lunes,Sábado,2017-02-01 08:30:00,2017-07-01 09:30:00
1,2017-01-05 09:06:10,2017-01-09 09:06:10,2017-01-05 09:06:10,2017-01-09 09:06:10,345600.0,2017-01-05,2017-01-09,Jueves,Lunes,2017-05-01 08:30:00,2017-09-01 08:30:00


In [119]:
hora_termino_lab = datetime.strptime('21:00:00', "%H:%M:%S").time()
z = pd.to_timedelta(datetime.combine(date.today(), hora_termino_lab) - datetime.combine(date.today(), x['CREATEDON_AUX'][0].time())).total_seconds()
z

42830.0

In [100]:
x["CREATEDON_AUX"][0].time()

datetime.time(9, 6, 10)

In [123]:
hora_termino_lab = datetime.strptime('21:00:00', "%H:%M:%S").time()

def one(y):
  tiempo_respuesta = pd.to_timedelta(y['UPDATEDON_AUX'] - y['CREATEDON_AUX']).dt.total_seconds()
  return tiempo_respuesta

def two(y):
  tiempo_respuesta = pd.to_timedelta(y['UPDATEDON_AUX'] - y['INICIO_CREATED']).dt.total_seconds()
  return tiempo_respuesta

def three(y):
  tiempo_respuesta = pd.to_timedelta(y['UPDATEDON_AUX'] - y['UPDATEDON_AUX']).dt.total_seconds()
  return tiempo_respuesta

def four(y):  
  print('Four')
  tiempo_respuesta = pd.to_timedelta(datetime.combine(date.today(), hora_termino_lab) - datetime.combine(date.today(), x['CREATEDON_AUX'][0].time())).total_seconds() + np.where(
      pd.to_timedelta(y['UPDATEDON_AUX'] - y['INICIO_ANSWERED']).total_seconds() < 0, 
           0, 
           pd.to_timedelta(y['UPDATEDON_AUX'] - y['INICIO_ANSWERED']).total_seconds())+ np.where(
               pd.to_timedelta(y['UPDATEDON_AUX'] - y['CREATEDON_AUX']).total_seconds() > 1,
               pd.to_timedelta(y["UPDATEDON_AUX"] - y['CREATEDON_AUX']).total_seconds()*11.5*60*60,
                0)

                                      #  difftime(as.ITime(hora_termino_lab),as.ITime(y$CREATEDON_AUX),units = 'secs')+

                                      #  ifelse(difftime(as.ITime(y$UPDATEDON_AUX),as.ITime(y$INICIO_ANSWERED),units = 'secs')<0,
                                      #         0,
                                      #         difftime(as.ITime(y$UPDATEDON_AUX),as.ITime(y$INICIO_ANSWERED),units = ' secs'))+
                                      #  ifelse(date(y$UPDATEDON_AUX) - date(y$CREATEDON_AUX)>1,
                                      #         (date(y$UPDATEDON_AUX) - date(y$CREATEDON_AUX))*11.5*60*60,
                                      #         0), 

  return tiempo_respuesta

def five(y):
  print('FIve')
  tiempo_respuesta = np.where(pd.to_timedelta(y['UPDATEDON_AUX'] - y['INICIO_ANSWERED']).dt.total_seconds() < 0, 
           0, 
           np.where(pd.to_timedelta(y['UPDATEDON_AUX'] - y['CREATEDON_AUX']).dt.total_seconds() == 0,
                    0,
                     pd.to_timedelta(y['UPDATEDON_AUX'] - y['INICIO_ANSWERED']).dt.total_seconds()) + np.where(
                        pd.to_timedelta(y['UPDATEDON_AUX'] - y['CREATEDON_AUX']).dt.total_seconds() > 1, pd.to_timedelta(y['UPDATEDON_AUX'] - y['CREATEDON_AUX']).dt.total_seconds()*11.5*60*60,0))
           
  return tiempo_respuesta


def perform_operation(y):

### SI ES EL MISMO DIA Y LA HORA DE CREACIÓN AJUSTADA ES MENOR A LA HORA DE INICIO TRABAJO, RESTO NORMAL 
  if (y["CREATEDON_AUX"] == y["UPDATEDON_AUX"] and y["INICIO_CREATED"] <= y["CREATEDON_AUX"] and y["CREATEDON_AUX"].time() < hora_termino_lab): 
    return one(y)

# SI ES EL MISMO DIA Y LA HORA DE CREACIÓN ES MENOR A LA HORA DE INICIO TRABAJO Y SE RESPONDE DESPUES DE LA HORA DE INGRESO, 
#RESTO DESDE LA HORA DE INICIO DEL DIA 
  if (y["CREATEDON_AUX"] == y["UPDATEDON_AUX"] and y["CREATEDON_AUX"] <= y["INICIO_CREATED"] and y["UPDATEDON_AUX"] > y["INICIO_CREATED"]): 
    return two(y)


# SI ES EL MISMO DIA Y LA HORA DE CREACIÓN ES MENOR A LA HORA DE INICIO TRABAJO Y SE RESPONDE ANTES DE LA HORA DE INGRESO, 
#RESTO NORMAL                         
  if (y["CREATEDON_AUX"] == y["UPDATEDON_AUX"] and y["CREATEDON_AUX"] <= y["INICIO_CREATED"] and y["UPDATEDON_AUX"] <= y["INICIO_CREATED"]):
    return three(y)                 
                         
                         
### SI ES OTRO DIA Y ANTES DE LA HORA DE TÉRMINO DEL DIA LABORAL, 
#RESTO EL TIEMPO REAL DE TRABAJO 
  if (y["CREATEDON_AUX"] <= y["UPDATEDON_AUX"] and y["CREATEDON_AUX"].time() <= hora_termino_lab): 
    return four(y) 


### SI ES OTRO DIA Y DESPUES DE LA HORA DE TÉRMINO DEL DIA LABORAL, RESTO EL TIEMPO REAL DE TRABAJO SOLO DEL OTRO DIA 
  if (y["CREATEDON_AUX"] <= y["UPDATEDON_AUX"] and y["CREATEDON_AUX"].time() > hora_termino_lab):
    return five(y)

In [110]:
y =  pd.to_timedelta(x["UPDATEDON_AUX"] - x['CREATEDON_AUX']).dt.total_seconds()
y

0    439430.0
1    345600.0
dtype: float64

In [124]:
x["tiempo_respuesta"] = x.apply(lambda y: perform_operation(y), axis=1)
x["tiempo_respuesta"]

Four
Four


In [126]:
x

,CREATEDON,UPDATEDON,CREATEDON_AUX,UPDATEDON_AUX,TiempoRespuesta_2,day_created,day_answer,DIA_SEMANA_CREATED,DIA_SEMANA_ANSWER,INICIO_CREATED,INICIO_ANSWERED,tiempo_respuesta
0,2017-01-02 09:06:10,2017-01-07 11:10:00,2017-01-02 09:06:10,2017-01-07 11:10:00,439430.0,2017-01-02,2017-01-07,Lunes,Sábado,2017-02-01 08:30:00,2017-07-01 09:30:00,1.819244e+10
1,2017-01-05 09:06:10,2017-01-09 09:06:10,2017-01-05 09:06:10,2017-01-09 09:06:10,345600.0,2017-01-05,2017-01-09,Jueves,Lunes,2017-05-01 08:30:00,2017-09-01 08:30:00,1.430788e+10


In [135]:
  y = x[['CREATEDON', 'UPDATEDON', 'tiempo_respuesta']].copy()
return y

,CREATEDON,UPDATEDON,tiempo_respuesta
0,2017-01-02 09:06:10,2017-01-07 11:10:00,1.819244e+10
1,2017-01-05 09:06:10,2017-01-09 09:06:10,1.430788e+10


In [ ]:
 # Añadir el tiempo de respuesta real.
x< - cbind(x,tiempo_respuesta= 
           ### SI ES EL MISMO DIA Y LA HORA DE CREACIÓN AJUSTADA ES MENOR A LA HORA DE INICIO TRABAJO, RESTO NORMAL 
           ifelse(date(x$CREATEDON_AUX)==date(x$UPDATEDON_AUX) & 
                  as.ITime(x$INICIO_CREATED)<=as.ITime( x$CREATEDON_AUX)& 
                  as.ITime(x$CREATEDON_AUX)<as.ITime('21:00:00'),
                  
                  difftime(as.ITime(x$UPDATEDON_AUX),as.ITime(x$UPDATEDON_AUX),units = 'secs'), 
                  # SI ES EL MISMO DIA Y LA HORA DE CREACIÓN ES MENOR A LA HORA DE INICIO TRABAJO Y SE RESPONDE DESPUES DE LA HORA DE INGRESO, 
                  #RESTO DESDE LA HORA DE INICIO DEL DIA 
                  ifelse(date(x$CREATEDON_AUX)==date(x$UPDATEDON_AUX) & 
                         as.ITime(x$CREATEDON_AUX)<=as.ITime(x$INICIO_CREATED) & 
                         as.ITime(x$UPDATEDON_AUX)>as.ITime(x$INICIO_CREATED) ,
                         
                         difftime(as.ITime(x$UPDATEDON_AUX),as.ITime(x$INICIO_CREATED),units = 'secs'), 
                         # SI ES EL MISMO DIA Y LA HORA DE CREACIÓN ES MENOR A LA HORA DE INICIO TRABAJO Y SE RESPONDE ANTES DE LA HORA DE INGRESO, 
                         #RESTO NORMAL 
                         ifelse(date(x$CREATEDON_AUX)==date(x$UPDATEDON_AUX) & 
                                as.ITime(x$CREATEDON_AUX)<=as.ITime(x$INICIO_CREATED) & 
                                as.ITime(x$UPDATEDON_AUX)<=as.ITime(x$INICIO_CREATED),
                                
                                difftime(as.ITime(x$UPDATEDON_AUX),as.ITime(x $UPDATEDON_AUX),units = 'sec s'), 
                                ### SI ES OTRO DIA Y ANTES DE LA HORA DE TÉRMINO DEL DIA LABORAL, 
                                #RESTO EL TIEMPO REAL DE TRABAJO 
                                ifelse(date(x$CREATEDON_AUX)<=date(x$UPDATEDON_AUX) & 
                                       as.ITime(x$CREATEDON_AUX)<= as.ITime('21:00:00'), 
                                       
                                       
                                       difftime(as.ITime('21:00:00'),as.ITime(x$CREATEDON_AUX),units = 'secs')+
                                       ifelse(difftime(as.ITime(x$UPDATEDON_AUX),as.ITime(x$INICIO_AWNSERED),units = 'secs')<0,
                                              0,
                                              difftime(as.ITime(x$UPDATEDON_AUX),as.ITime(x$INICIO_AWNSERED),units = ' secs'))+
                                       ifelse(date(x$UPDATEDON_AUX) - date(x$CREATEDON_AUX)>1,
                                              (date(x$UPDATEDON_AUX) - date(x$CREATEDON_AUX))*11.5*60*60,
                                              0), 
                                       ### SI ES OTRO DIA Y DESPUES DE LA HORA DE TÉRMINO DEL DIA LABORAL, RESTO EL TIEMPO REAL DE TRABAJO SOLO DEL OTRO DIA 

                                       ifelse(date(x$CREATEDON_AUX)<=date(x$UPDATEDON_AUX) & as.ITime(x$CREATEDON_AUX)>as.ITime('21:00:00'),
                                              
                                               
                                              ifelse(difftime(as.ITime(x$UPDATEDON_AUX),as.ITime(x$INICIO_AWNSERED),units = 'secs' )<0,0,
                                                     ifelse(date(x$UPDATEDON_AUX) - date(x$CREATEDON_AUX)==0,0,
                                                            difftime(as.ITime(x$UPDATEDON_AUX),as.ITime(x$INICIO_AWNSERED),units = 'secs'))+
                                                     ifelse(date(x$UPDATEDON_AUX) - date(x$CREATEDON_AUX)>1,(date(x$UPDATEDON_AUX) - date(x$CREATEDON_AUX))*11.5*60*60,0)), NA )))))) 